# 如何流式传输 LLM 的响应

所有 `LLM` 都实现了 [Runnable 接口](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable)，该接口带有标准可运行方法的**默认**实现（即 `ainvoke`、`batch`、`abatch`、`stream`、`astream`、`astream_events`）。

**默认**的流式传输实现提供了一个 `Iterator`（对于异步流式传输则是 `AsyncIterator`），它只 yields 一个值：底层聊天模型提供商的最终输出。

逐个令牌流式传输输出的能力取决于提供商是否实现了适当的流式传输支持。

在此处查看哪些 [集成支持令牌逐个流式传输](/docs/integrations/llms/)。

:::note

**默认**实现**不**支持令牌逐个流式传输，但它确保模型可以替换为任何其他模型，因为它支持相同的标准接口。

:::

## 同步流

下方我们使用 `|` 来帮助可视化 token 之间的分隔符。

In [1]:
from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0, max_tokens=512)
for chunk in llm.stream("Write me a 1 verse song about sparkling water."):
    print(chunk, end="|", flush=True)



|Spark|ling| water|,| oh| so clear|
|Bubbles dancing|,| without| fear|
|Refreshing| taste|,| a| pure| delight|
|Spark|ling| water|,| my| thirst|'s| delight||

## 异步流

让我们看看如何使用 `astream` 在异步环境中进行流式传输。

In [2]:
from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0, max_tokens=512)
async for chunk in llm.astream("Write me a 1 verse song about sparkling water."):
    print(chunk, end="|", flush=True)



|Spark|ling| water|,| oh| so clear|
|Bubbles dancing|,| without| fear|
|Refreshing| taste|,| a| pure| delight|
|Spark|ling| water|,| my| thirst|'s| delight||

## 异步事件流


LLM 也支持标准的 [astream events](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable.astream_events) 方法。

:::tip

当在包含多个步骤的大型 LLM 应用（例如涉及 `agent` 的应用）中实现流式传输时，`astream_events` 非常有用。
:::

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0, max_tokens=512)

idx = 0

async for event in llm.astream_events(
    "Write me a 1 verse song about goldfish on the moon", version="v1"
):
    idx += 1
    if idx >= 5:  # Truncate the output
        print("...Truncated")
        break
    print(event)